# LLM Fraud Explanation Demo

This notebook demonstrates how engineered transaction features and
model outputs are converted into human-readable fraud explanations
using an LLM-based explanation layer.


In [12]:
import pandas as pd
import numpy as np

from src.llm_explainer import FraudLLMExplainer


In [13]:
df = pd.read_parquet("transactions_features.parquet")

print(df.shape)
df.head()


(118108, 446)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,card_amt_mean,card_amt_std,amt_zscore,card_txn_count_day,card_amt_sum_day,time_since_prev_txn,missing_identity_count,has_identity_info,high_amount_flag,night_transaction_flag
0,3021401,0,850730,23.203,C,1007,555.0,135.0,mastercard,224.0,...,39.183500,22.599840,-0.707107,1,23.203,-1.0,19,1,0,0
1,3226241,0,5651177,55.164,C,1007,555.0,135.0,mastercard,224.0,...,39.183500,22.599840,0.707107,1,55.164,4800447.0,19,1,0,0
2,3232272,0,5845013,57.950,W,1008,555.0,150.0,visa,226.0,...,57.950000,NaN,NaN,1,57.950,-1.0,38,0,0,0
3,3069808,0,1784656,482.350,W,1012,479.0,150.0,visa,162.0,...,272.433333,181.948841,1.153713,1,482.350,-1.0,38,0,0,0
4,3098185,0,2166984,175.000,R,1012,479.0,150.0,visa,162.0,...,272.433333,181.948841,-0.535499,1,175.000,382328.0,10,1,0,1


In [25]:
row = df[
    (df["isFraud"] == 1) &
    (df["missing_identity_count"] > 5) &
    (df["amt_zscore"].abs() > 1.5)
].sample(1, random_state=42).iloc[0]


In [26]:
features = {
    "amt_zscore": float(fraud_row["amt_zscore"]),
    "card_txn_count_day": int(fraud_row["card_txn_count_day"]),
    "card_amt_sum_day": float(fraud_row["card_amt_sum_day"]),
    "time_since_prev_txn": float(fraud_row["time_since_prev_txn"]),
    "missing_identity_count": int(fraud_row["missing_identity_count"]),
    "high_amount_flag": int(fraud_row["high_amount_flag"]),
    "night_transaction_flag": int(fraud_row["night_transaction_flag"])
}

features


{'amt_zscore': 0.0,
 'card_txn_count_day': 1,
 'card_amt_sum_day': 60.0,
 'time_since_prev_txn': 9506872.0,
 'missing_identity_count': 6,
 'high_amount_flag': 0,
 'night_transaction_flag': 0}

In [27]:
fraud_score = 0.87


In [28]:
for k, v in features.items():
    print(f"{k}: {v}")


amt_zscore: 0.0
card_txn_count_day: 1
card_amt_sum_day: 60.0
time_since_prev_txn: 9506872.0
missing_identity_count: 6
high_amount_flag: 0
night_transaction_flag: 0


In [29]:
explainer = FraudLLMExplainer()

explanation = explainer.explain(features, fraud_score)

print(explanation)


This transaction was flagged with a fraud probability of 0.87 because important device or identity information is missing.


### Explanation Interpretation

The explanation highlights abnormal spending behavior, transaction velocity,
and missing identity signals. This mirrors how fraud analysts interpret alerts
in real-world financial institutions, where model transparency and reasoning
are essential for investigation and compliance.
